In [26]:
import csv
import h5py
import os
import numpy as np

In [28]:
buggy_stims = "pilot-containment-cone-plate_0017 \
pilot-containment-cone-plate_0022 \
pilot-containment-cone-plate_0029 \
pilot-containment-cone-plate_0034 \
pilot-containment-multi-bowl_0042 \
pilot-containment-multi-bowl_0048 \
pilot-containment-vase_torus_0031 \
pilot_dominoes_SJ020_d3chairs_o1plants_tdwroom_0005 \
pilot_it2_collision_non-sphere_box_0002 \
pilot_it2_collision_non-sphere_tdw_1_dis_1_occ_0004 \
pilot_it2_collision_non-sphere_tdw_1_dis_1_occ_0007 \
pilot_it2_drop_simple_box_0000 \
pilot_it2_drop_simple_box_0042 \
pilot_it2_drop_simple_tdw_1_dis_1_occ_0003 \
pilot_it2_rollingSliding_simple_collision_box_0008 \
pilot_it2_rollingSliding_simple_collision_box_large_force_0009 \
pilot_it2_rollingSliding_simple_collision_tdw_1_dis_1_occ_0002 \
pilot_it2_rollingSliding_simple_ledge_tdw_1_dis_1_occ_sphere_small_zone_0022 \
pilot_it2_rollingSliding_simple_ramp_box_small_zone_0006 \
pilot_it2_rollingSliding_simple_ramp_tdw_1_dis_1_occ_small_zone_0004 \
pilot_it2_rollingSliding_simple_ramp_tdw_1_dis_1_occ_small_zone_0017 \
pilot_linking_nl1-8_mg000_aCyl_bCyl_tdwroom1_long_a_0022 \
pilot_linking_nl1-8_mg000_aCylcap_bCyl_tdwroom1_0012 \
pilot_linking_nl1-8_mg000_aCylcap_bCyl_tdwroom_small_rings_0006 \
pilot_linking_nl1-8_mg000_aCylcap_bCyl_tdwroom_small_rings_0010 \
pilot_linking_nl1-8_mg000_aCylcap_bCyl_tdwroom_small_rings_0029 \
pilot_linking_nl1-8_mg000_aCylcap_bCyl_tdwroom_small_rings_0036 \
pilot_linking_nl6_aNone_bCone_occ1_dis1_boxroom_0028 \
pilot_towers_nb4_fr015_SJ000_gr000sph_mono1_dis0_occ0_tdwroom_stable_0000 \
pilot_towers_nb4_fr015_SJ000_gr000sph_mono1_dis0_occ0_tdwroom_stable_0002 \
pilot_towers_nb4_fr015_SJ000_gr000sph_mono1_dis0_occ0_tdwroom_stable_0003 \
pilot_towers_nb4_fr015_SJ000_gr000sph_mono1_dis0_occ0_tdwroom_stable_0010 \
pilot_towers_nb4_fr015_SJ000_gr000sph_mono1_dis0_occ0_tdwroom_stable_0013 \
pilot_towers_nb4_fr015_SJ000_gr000sph_mono1_dis0_occ0_tdwroom_stable_0017 \
pilot_towers_nb4_fr015_SJ000_gr000sph_mono1_dis0_occ0_tdwroom_stable_0018 \
pilot_towers_nb4_fr015_SJ000_gr000sph_mono1_dis0_occ0_tdwroom_stable_0032 \
pilot_towers_nb4_fr015_SJ000_gr000sph_mono1_dis0_occ0_tdwroom_stable_0036 \
pilot_towers_nb4_fr015_SJ000_gr01_mono0_dis1_occ1_tdwroom_unstable_0021 \
pilot_towers_nb4_fr015_SJ000_gr01_mono0_dis1_occ1_tdwroom_unstable_0041 \
pilot_towers_nb5_fr015_SJ030_mono0_dis0_occ0_boxroom_unstable_0006 \
pilot_towers_nb5_fr015_SJ030_mono0_dis0_occ0_boxroom_unstable_0009".split(' ')

In [38]:
data = [['Stimulus Name', 'Scenario', 'Predicted Outcome', 'Ground Truth', 'Correct', 'Scenario Accuracy', 'Full Scenario Accuracy', 'Test Accuracy']]
all_scenarios = ['collision', 'drop', 'towers', 'link', 'roll', 'contain', 'dominoes']
all_scenarios_map = {'collision':[], 'drop':[], 'towers':[], 'link':[], 'roll':[], 'contain':[], 'dominoes':[]}



h5_data = h5py.File('/ccn2/u/thekej/models/sgnn_physion/ocd/test_features.hdf5', 'r')
h5_filename = [f.decode().split('/')[-1] for f in h5_data['filenames'][:]]
h5_labels = h5_data['label']

with open('annotations.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in spamreader:
        stim = row[0][:-4]
        if stim in buggy_stims:
            continue
        pred = True if row[1] == 'Contact' else False
        file_scenario = next((sc for sc in all_scenarios if sc in stim), 'unknown')
        idx = h5_filename.index(stim)
        gt = h5_labels[idx]
        correct = int(gt == pred)
        data += [[stim, file_scenario, pred, gt, correct]]
        all_scenarios_map[file_scenario] += [correct]

full_sce_mean = 0

for sc in all_scenarios:
    full_sce_mean += np.mean(all_scenarios_map[sc])

full_sce_mean = full_sce_mean / len(all_scenarios)

test_mean = 0
for d in data[1:]:
    test_mean += d[4]
test_mean = test_mean / len(data[1:])

for i in range(len(data)):
    if i == 0:
        continue
    data[i].append(np.mean(all_scenarios_map[data[i][1]]))
    data[i].append(full_sce_mean)
    data[i].append(test_mean)
    
with open('annotations_khaled.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for row in data:
        spamwriter.writerow(row)
        
h5_data.close()        